In [68]:
import sys # for automation and parallelization
manual, scenario = (True, 'base') if 'ipykernel' in sys.argv[0] else (False, sys.argv[1])

# START

In [69]:
import sys
sys.path.insert(0, r'../../../quetzal')
from quetzal.model import stepmodel
from quetzal.io import excel
import pandas as pd
if manual:
    %matplotlib inline

In [70]:
training_folder = '../../'
plot_path = training_folder + 'outputs/plot/'

In [71]:
distribution = stepmodel.read_zippedpickles(training_folder + r'model/{scen}/distribution'.format(scen=scenario))
pt = stepmodel.read_zippedpickles(training_folder + r'model/{scen}/pruning_pt_pathfinder'.format(scen=scenario))
car = stepmodel.read_zippedpickles(training_folder + r'model/{scen}/constrained_road_pathfinder'.format(scen=scenario))

zone_to_transit: 100%|█████████████████████████████████████████████████████████████████| 33/33 [00:00<00:00, 37.65it/s]


In [72]:
sm = pt.copy()
sm.car_los = car.car_los
sm.volumes = distribution.volumes
# each segment will be processed independently in the logit
sm.segments = ['car_owner', 'pt_captive'] 

In [73]:
sm.analysis_pt_los(walk_on_road=True)

path_analysis: 100%|██████████████████████████████████████████████████████████| 51395/51395 [00:01<00:00, 34913.20it/s]


In [74]:
sm.analysis_pt_time(walk_on_road=False)


In [75]:
# the scenario specific variables are read from the parameter file
var = excel.read_var(file='../../inputs/parameters.xlsx', scenario=scenario,
                               )

var.head()

category               parameter        
general                parent               base
                       bus_speed_factor      0.8
                       boarding_cost_acf     300
                       boarding_cost_wor     300
preparation_footpaths  max_length           3000
Name: base, dtype: object

In [76]:
kwargs = var['preparation_logit'].to_dict()

# Parameters conversion
kwargs

{'time': -0.00055,
 'price': -1,
 'transfers': -0.25,
 'mode': 1,
 'pt_mode': 0.5,
 'pt_path': 0.1}

# LOGIT SCALES

In [77]:
sm.preparation_logit(
    segments=['car_owner', 'pt_captive'],
    **kwargs
)

In [78]:
sm.utility_values.T

value,time,price,ntransfers,mode_utility
segment,,,,
root,-0.00055,-1.0,-0.25,1.0
car_owner,-0.00055,-1.0,-0.25,1.0
pt_captive,-0.00055,-1.0,-0.25,1.0


In [79]:
sm.logit_scales.T # phi

route_type,rail,walk,car,bus,root,subway,tram,pt
segment,,,,,,,,
root,0.1,0.0,0.0,0.1,1.0,0.1,0.1,0.5
car_owner,0.1,0.0,0.0,0.1,1.0,0.1,0.1,0.5
pt_captive,0.1,0.0,0.0,0.1,1.0,0.1,0.1,0.5


In [80]:
sm.mode_utility.loc[ 'car', 'pt_captive'] = -100 # PT captive can not use the car
sm.mode_utility.T

route_type,rail,walk,car,bus,root,subway,tram
segment,,,,,,,
root,0,0,0,0,0,0,0
car_owner,0,0,0,0,0,0,0
pt_captive,0,0,-100,0,0,0,0


In [81]:
sm.mode_nests.T # nested structure can be set for each segment

route_type,rail,walk,car,bus,root,subway,tram,pt
segment,,,,,,,,
root,pt,root,root,pt,NaN,pt,pt,root
car_owner,pt,root,root,pt,NaN,pt,pt,root
pt_captive,pt,root,root,pt,NaN,pt,pt,root


# UTILITY EVALUATION
## add relevant columns to path dataframes

In [82]:
for segment in sm.segments:
    segment_utility = var.loc[(segment,sm.mode_utility.index)].droplevel(level=0)
    sm.mode_utility.loc[segment_utility.index,segment] = segment_utility.to_list()
    
sm.mode_utility = sm.mode_utility.fillna(0)
sm.mode_utility = sm.mode_utility*60*sm.utility_values.loc['time']

print(sm.mode_utility)

segment     root  car_owner  pt_captive
route_type                             
rail        -0.0    -0.1485     -0.0165
walk        -0.0    -0.0000     -0.0000
car         -0.0    -0.3300     -0.0000
bus         -0.0    -0.3960     -0.0495
root        -0.0    -0.0000     -0.0000
subway      -0.0    -0.0330     -0.0495
tram        -0.0    -0.0000     -0.0000


In [83]:

sm.pt_los['price'] = 0 # sm.analysis_pt_fare
sm.car_los['ntransfers'] = 0
sm.car_los['price'] = 0

In [84]:
sm.pt_los

,origin,destination,gtime,path,pathfinder_session,reversed,route_id_set,route_type_set,broken_column,broken_route_type,...,all_walk,ntransfers,time_link_path,length_link_path,access_time,footpath_time,in_vehicle_time,waiting_time,boarding_time,price
0,zone_0,zone_0,-1.000000e+09,"(zone_0,)",best_path,False,{},{},NaN,NaN,...,True,0,0.0,0.0,0.000000,0.0,0.0,0.0,0,0
1,zone_0,zone_1,1.111811e+03,"(zone_0, 45, link_4303, 135, zone_1)",best_path,False,{1},{subway},NaN,NaN,...,False,0,60.0,NaN,999.811375,0.0,60.0,52.0,0,0
2,zone_0,zone_10,1.445932e+03,"(zone_0, 187, link_3656, 209, link_4524, 110, ...",best_path,False,"{95, 4}","{bus, subway}",NaN,NaN,...,False,1,240.0,NaN,988.931956,0.0,240.0,217.0,0,0
3,zone_0,zone_100,1.788474e+03,"(zone_0, 45, link_4303, link_4304, 26, link_49...",best_path,False,"{7B, 1}",{subway},NaN,NaN,...,False,3,600.0,NaN,890.473793,0.0,600.0,298.0,0,0
4,zone_0,zone_101,2.387717e+03,"(zone_0, 45, link_4303, link_4304, 26, link_44...",best_path,False,"{5, 1}","{subway, rail}",NaN,NaN,...,False,4,660.0,NaN,1400.217287,0.0,660.0,327.5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,zone_58,zone_107,2.419927e+03,"(zone_58, 146, link_5244, 29, link_5382, link_...",route_breaker,True,NaN,NaN,route_id,NaN,...,False,4,1080.0,NaN,798.427098,0.0,1080.0,541.5,0,0
110,zone_109,zone_83,2.456258e+03,"(zone_109, 270, link_1765, 98, link_4465, link...",route_breaker,True,NaN,NaN,route_id,NaN,...,False,5,1020.0,NaN,740.258445,0.0,1020.0,696.0,0,0
123,zone_70,zone_83,2.474395e+03,"(zone_70, 34, link_4875, link_4876, link_4877,...",route_breaker,True,NaN,NaN,route_id,NaN,...,False,2,900.0,NaN,1137.394945,0.0,900.0,437.0,0,0
127,zone_74,zone_83,2.535796e+03,"(zone_74, 289, link_4874, link_4875, link_4876...",route_breaker,True,NaN,NaN,route_id,NaN,...,False,2,960.0,NaN,1138.796295,0.0,960.0,437.0,0,0


## calculate utility

In [85]:
#on à plus le detail dans le car_los semble il

In [86]:
sm.analysis_pt_route_type(hierarchy=[ 'car','rail', 'subway', 'tram', 'bus', 'walk'])
sm.analysis_car_route_type() # add route_type = 'car' in car_los

# concatenate pt_los and car_los
sm.los = pd.concat([sm.pt_los, sm.car_los]).reset_index(drop=True)
sm.los['path'] = sm.los['path'].apply(lambda p: tuple(p))
sm.analysis_mode_utility(how=var['analysis_mode_utility']['how'])

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 32.81it/s]


## calulate the utilities and the probabilities

In [87]:
sm.step_logit(n_paths_max=10, workers=1, nchunks=10)

sm.compute_los_volume()

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  6.27it/s]


In [88]:
sm.step_assignment(
        road=False, 
        boardings=True, 
        alightings=True, 
        transfers=True,
        segmented=False,
        boarding_links=True,
        alighting_links=True)

# ASSIGNMENT

In [89]:
sm.pt_los = sm.los.copy()
sm.pt_los = sm.pt_los.loc[sm.pt_los['route_type'] != 'car']
sm.step_assignment(
    road=True,
    boardings=True,
    boarding_links=True,
    alightings=True,
    alighting_links=True,
    transfers=True,
)

# XLSX EXPORT

In [91]:
sm.summary_link_max(inplace=True)
sm.summary_link_sum(inplace=True)
#sm.summary_od(inplace=True)

#sm.summary_path_average(inplace=True)
sm.summary_aggregated_path_average(inplace=True, pt_route_types=['subway', 'bus', 'tram', 'rail'])
sm.summary_path_sum(inplace=True)

s = sm.stack_link_sum.unstack([-1, -2]).groupby(level=0).sum().stack([0, 1])
s.name = 'sum'
sm.stack_aggregated_link_sum = s 
s = sm.stack_link_max.unstack().groupby(level=0).max().stack()
s.name = 'max'
sm.stack_aggregated_link_max = sOka

KeyError: 'in_vehicle_length'

In [ ]:
sm.od_los.head(2)

In [ ]:
sm.to_excel(training_folder + r'model/%s/stacks.xlsx'%scenario, prefix='stack')

# GEOJSON EXPORT

In [ ]:
if manual:
    sm.to_json(training_folder + 'json', only_attributes=['links', 'nodes'])

# ZIP EXPORT

In [ ]:
sm.to_zip(training_folder + 'model/{scen}/logit.zip'.format(scen=scenario), omitted_attributes=['los'])

# PLOT

In [ ]:
import numpy as np
paths = sm.pt_los
link_routes = sm.links['route_short_name'].to_dict()
def link_path_to_routes(link_path):
    routes = [link_routes[l] for l in link_path]
    used = set()
    rsn = []
    for route in routes:
        if route not in used:
            rsn.append(route)
        used.add(route)
    return rsn

paths['route_path'] = paths['link_path'].apply(link_path_to_routes)
paths['ntransfers'] = paths['transfers'].apply(len)

def title(paths):
    paths['min'] = np.round(paths['time'] / 60).astype(int)
    mins = (paths[['in_vehicle_time', 'access_time', 'footpath_time', 'waiting_time', 'time']] / 60).astype(int).astype(str)
    title = 'P = ' + (np.round(paths[('pt_captive', 'probability')], 2).fillna(0) * 100).astype(int).astype(str) + '%'
    title += ' | '+  paths['ntransfers'].astype(str) + ' transfers '
    title += ' | '+  paths['min'].astype(str) + ' mins | '
    title += paths['route_path'].apply(lambda l: '—>'.join(l) )
    title += '\n'+ 'in vehicle '+ mins['in_vehicle_time']+ "' | " + ' waiting '+ mins['waiting_time']+ "' | "  
    title += 'access ' + mins['access_time']+ "' | "   + ' footpaths '  + mins['footpath_time']+ "'"   
    return title


paths['title'] = title(paths)

In [ ]:
if manual:
    origin, destination='zone_74',  'zone_105'
    n =  len(sm.pt_los.set_index(['origin', 'destination']).loc[(origin, destination)])

    rows = 2
    if n > 6:
        rows = 3
    if n > 12:
        rows = 4

    columns = n // rows + bool(n%rows)


    url = 'http://a.basemaps.cartocdn.com/light_nolabels/{z}/{x}/{y}.png'
    fig, axes = sm.plot_separated_paths(
        origin, destination, rows=rows, figsize=[20, 20],
        title='title', constrained_layout=True,
        basemap_url=url, zoom=13
    )
    ax = fig.get_axes()[0]
    bbox = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    fig.set_size_inches(bbox.width*columns, rows*(bbox.height))
    fig.constrained_layout = True

In [ ]:
if manual:
    from syspy.syspy_utils import data_visualization as dv
    bandwidth = dv.bandwidth
    power=0.5
    lv = [100, 1000, 2000, 5000, 7000, 15000]
    url = 'http://a.basemaps.cartocdn.com/dark_nolabels/{z}/{x}/{y}.png'

    sm.road_links['volume'] = sm.road_links[('volume', 'pt')]
    i_links = sm.links.loc[sm.links.road_length.isnull()]
    df = pd.concat([i_links, sm.road_links])
    df = df.loc[df['volume'] > 0]

    plot= bandwidth(
        df, value_column='volume', power=power, figsize=[12, 12], legend_values=lv,
        label_kwargs={'color':'white', 'fontsize':12}
    )
    dv.add_basemap(plot, url=url, zoom=12)
    plot.set_title('Public Transport Assignment on Bus Lines (Stochastic)', fontsize=16)